In [1]:
import sys
sys.path.append("..")
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
from writeit import rewritePipes
%pylab inline

ImportError: No module named numpy

In [ ]:
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)

In [ ]:
mtype =1
Np=2

Ds = [1.]*Np
jt = [1,2,1]
bt = [1,1,1]
bv = [0,0,0]
r =  [-1,0,1]
h0s =[0.]*Np
q0s =[0.]*Np

Ls = [100.,100]
Ns = [int(L) for L in Ls]
T = 60
M = 7200
Mi =10
a = 100
elevs = [0,0,0]
H = []
N = Ns[0]
dt = T/float(M)
A0 = 0.792325375237 #corresponds to H = 10(verify?)
Qin = A0*np.ones(M+1)
a0 = A0*np.ones(N+1)
q0 = np.ones(N+1)
q1 = np.zeros(N+1)
fn = "../indata/testfill"
oldinp = "../indata/2pipes.inp"
Mrtest=[0.00, 0.008, 0.015]
for jj in range(3):
    Mr = Mrtest[jj]
    Mrs = [Mr]*Np
    for ii in range(5):
        a1 = .1*ii*np.ones(N+1)
        (fi, fc) = rewritePipes(fn,oldinp, Ns, Ls, Mrs, Ds, jt, bt, bv, r, h0s, q0s, T, M, a,elevs)
        n0 = PyNetwork(fi, fc, mtype)
        n0.setIC(0,a0,q0)
        n0.setIC(1,a1,q1)
        n0.setbVal(0,Qin)
        V0 = n0.getTotalVolume()
        n0.runFo
        rwardProblem(dt) 
        print "Manning coeff = %.3f and a0/Af = %.3f"%(Mr,a1[0])
        print "Inflow Volume = %.2f"%(n0.getTotalVolume()-V0)
        p0 = PyPipe_ps(Ns[0], Ds[0], Ls[0], M, a)
        qf = n0.qhist(1)
        Nstar = 10
        H.append([p0.pbar(qf[idx_t(0,Nstar,n,Ns[0])],False) for n in range(M+1)])

In [ ]:
fig, ax = plt.subplots(figsize = (10,10),nrows=3)
from matplotlib import cm
import matplotlib.colors as colors
cmap = cm.get_cmap('ocean')
cnorm =colors.Normalize(vmin=0, vmax = len(H)+1)
scalarMap = cm.ScalarMappable(norm =cnorm, cmap= cmap)
t = np.linspace(0,n0.T,n0.M+1)
for j in range(3):    
    for kk in range(5):
        cval = scalarMap.to_rgba(kk)
        ax[j].plot(t,H[kk],color = cval, label = "p0 = %.2f m"%p0.pbar(.1*kk,False))
    legend(loc = "upper left")
    ax[j].set_title("Manning coeff = %.3f"%Mrtest[j])
    ax[j].set_ylim(0,25)
    ax[j].set_ylabel(r'$\bar{p}$ (m)')
    ax[j].set_xlabel('t')

In [ ]:
print p0.AofH(10,True)
print p0.HofA(A0,True)

In [ ]:
p2 = [p0.pbar(qf[idx_t(0,k,500,Ns[0])],False) for k in range(N)]
x = linspace(0,Ls[0],N)
plot(x,p2)

In [ ]:
pldx = Ls[0]/Ns[0]

In [ ]:
print n0.cmax[0]*dt/dx

In [ ]:
print N